In [1]:
# -*- coding: utf-8 -*-

#Convert mutliclasses into Single Class

#Resolution: 0.004

#Created on Mon Apr 11 16:42:38 2022

#@author: li.chao.987@s.kyushu-u.ac.jp


from osgeo import gdal
import numpy as np
import os

In [2]:
def createSingleClassTiff (year, aimFolder):
    original_raster = gdal.Open("F:\\17_Article\\01_Data\\01_LandCover\\LandCoverMerge\\LandCover_UMD_" + str(year) + "_500m.tif")
    #print(original_raster)

    raster_rprj = gdal.Warp(aimFolder + "LandCover_UMD_"+ str(year) +"_0004_WGS.tif", original_raster, dstSRS = "EPSG:4326", xRes = 0.004, yRes = 0.004, resampleAlg = "mode")
    print(raster_rprj)
    #raster_rprj = None

    original_raster = None

    band = raster_rprj.GetRasterBand(1)

    multiLista = band.ReadAsArray()

    # reclassification
    singleRasterLocation = "F:/17_Article/01_Data/01_LandCover/LandCoverSingleClass/"
    driver = gdal.GetDriverByName("GTiff")
    aim = 0
    while aim < 16:
        print(aim)
        originalToSingleLista = multiLista.copy()
        orignialToSingleListaClip = originalToSingleLista[:,37750*2:41750*2].copy()
        #orignialToSingleListaClip = 
        orignialToSingleListaClip[orignialToSingleListaClip == aim] = 100
        orignialToSingleListaClip[orignialToSingleListaClip != 100] = 0
        
        #for j in range(raster_rprj.RasterXSize):
        #    for i in range(raster_rprj.RasterYSize):
        #        if originalToSingleLista[i, j] == aim:
        #            originalToSingleLista[i, j] = 1
        #        else:
        #            originalToSingleLista[i, j] = 0
                    
        # create new file
        name = "year_" + str(year) + "_class_" + str(aim) + ".tif"
        singleRaster = driver.Create(singleRasterLocation + name, 8000, 7500, 1)
        singleRaster.GetRasterBand(1).WriteArray(orignialToSingleListaClip)
        
        proj = raster_rprj.GetProjection()
        georef = (122.0, 0.004, 0.0, 50.0, 0.0, -0.004)
        singleRaster.SetProjection(proj)
        singleRaster.SetGeoTransform(georef)
        singleRaster.FlushCache()
        singleRaster = None
        aim += 1
        
    raster_rprj = None   

    os.remove(aimFolder + "LandCover_UMD_" + str(year) + "_0004_WGS.tif")


In [14]:
aimFolder = "F:\\17_Article\\01_Data\\01_LandCover\\LandCover0004WGS\\"
#os.mkdir("F:\\17_Article\\01_Data\\01_LandCover\\LandCover0004WGS")

year=2001
# to 2020

import multiprocessing

multiprocessing.cpu_count()

12

In [15]:
from joblib import Parallel, delayed
import multiprocessing

Parallel(n_jobs=8)(delayed(createSingleClassTiff)(int(year), aimFolder) for year in np.linspace(2001, 2020, 20))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [16]:
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
from scipy.spatial import cKDTree

# make the raster -180 -60 180 90
nx = 8000                                       # number of cells in the x direction
ny = 7500                                     # number of cells in the y direction
xmin = 122.002                                  # x coordinate of lower, left cell center 
ymin = 30.002                                     # y coordinate of lower, left cell center 
xsize = 0.004                                   # extent of cells in x direction
ysize = 0.004                                   # extent of cells in y direction
epsg = 4326                                     # get projection

def addCoord(nx,xmin,xsize,ny,ymin,ysize, epsg): # Michael Pyrcz, March, 2018                      
  # makes a 2D dataframe with coordinates based on GSLIB specification
  coords = np.zeros([nx*ny, 3])
  ixy = 0
  for iy in range(nx):
    for ix in range(ny):
      coords[ixy,1] = xmin + ix*xsize  
      coords[ixy,2] = ymin + iy*ysize 
      coords[ixy,0] = ixy + 1
      ixy = ixy + 1
      
  coords = pd.DataFrame(coords, columns = ["order", "X", "Y"])
  gdf = gpd.GeoDataFrame(coords, geometry = gpd.points_from_xy(coords.X, coords.Y))
  gdf = gdf.set_crs(epsg = epsg)
  return (gdf)

def ckdnearest(gdA, gdB, column_name):

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name=column_name)
        ], 
        axis=1)

    return gdf

coords_distance_raster = addCoord(nx,xmin,xsize,ny,ymin,ysize, epsg)

In [17]:
coords_extration = gpd.read_file("F:/17_Article/01_Data/00_mesh/Mesh500/mergedPointMesh500m.shp")
coords_distance_raster_result = coords_extration.copy()

In [22]:
# https://www.arcgis.com/home/item.html?id=dabc265276a5402a8645834e9a3365f0
# the data from Esri

japanBoundary = gpd.read_file("F:/17_Article/01_Data/00_mesh/JPN_Country_03degree.shp")
japanBoundary['inBuffer'] = 1
japanBoundary = japanBoundary[['inBuffer', 'geometry']]
japanBoundary.head()


,inBuffer,geometry
0,1,"MULTIPOLYGON (((15512009.759 5052211.941, 1551..."


In [24]:
join_coords_distance_raster = coords_distance_raster.sjoin(japanBoundary.to_crs(epsg = 4326), how='left')
join_coords_distance_raster.head()

,order,X,Y,geometry,index_right,inBuffer
0,1.0,122.002,30.002,POINT (122.00200 30.00200),NaN,NaN
1,2.0,122.006,30.002,POINT (122.00600 30.00200),NaN,NaN
2,3.0,122.010,30.002,POINT (122.01000 30.00200),NaN,NaN
3,4.0,122.014,30.002,POINT (122.01400 30.00200),NaN,NaN
4,5.0,122.018,30.002,POINT (122.01800 30.00200),NaN,NaN


In [27]:

filtered_coords_distance_raster = join_coords_distance_raster[join_coords_distance_raster["inBuffer"] == 1]

In [29]:
coords_distance_raster = None
join_coords_distance_raster = None

In [30]:
import gc

gc.collect()

203

In [31]:
del japanBoundary
gc.collect()

36

In [32]:
def buildClassAndDistanceDf(year, coords_extration, coords_distance_raster, coords_distance_raster_result):
    type_num = 0
    while type_num < 16:
    
        rasterLocation = "F:/17_Article/01_Data/01_LandCover/LandCoverSingleClass/year_" + str(year) + "_class_"+ str(type_num) +".tif"
        
        rasterFile = rasterio.open(rasterLocation)
        rasterArray = rasterFile.read(1)
        
        valueArray = []
        for point in coords_extration['geometry']:
            x = point.xy[0][0]
            y = point.xy[1][0]
            row, col = rasterFile.index(x, y)
            valueArray.append(rasterArray[row, col])
            
        valueArray = np.array(valueArray)
        coords_extration['value'] = valueArray
        coords_extration_aim = coords_extration[coords_extration.value == 100]
        coords_extration_aim = coords_extration_aim[['geometry']]
        
        if coords_extration_aim.shape[0] > 0:
            dist = ckdnearest(coords_distance_raster, coords_extration_aim, "dist_class_"+str(type_num))
            dist_column = dist[["dist_class_"+str(type_num)]]
            coords_distance_raster_result["binary_class_" + str(type_num) + "_" + str(year)] = valueArray / 100
            coords_distance_raster_result["dist_class_" + str(type_num) + "_" + str(year)] = dist_column
        else:
            print("class_"+str(type_num)+" is not in japan")
        print("dist_class_"+str(type_num))
        type_num += 1
    return coords_distance_raster_result

In [ ]:
from joblib import Parallel, delayed
import multiprocessing

Parallel(n_jobs=5)(delayed(buildClassAndDistanceDf)(int(year), coords_extration, filtered_coords_distance_raster, coords_distance_raster_result) for year in np.linspace(2001, 2020, 20))

In [9]:
# note: there is no open shrublands

#for type_num in range(16):
year = 2001
type_num = 0
while type_num < 16:
    
    rasterLocation = "F:/17_Article/01_Data/01_LandCover/LandCoverSingleClass/year_" + str(year) + "_class_"+ str(type_num) +".tif"
    
    rasterFile = rasterio.open(rasterLocation)
    aimPoint = coords_extration
    rasterArray = rasterFile.read(1)
    
    valueArray = []
    for point in aimPoint['geometry']:
        x = point.xy[0][0]
        y = point.xy[1][0]
        row, col = rasterFile.index(x, y)
        valueArray.append(rasterArray[row, col])
        
    valueArray = np.array(valueArray)
    coords_extration['value'] = valueArray
    coords_extration_aim = coords_extration[coords_extration.value == 100]
    coords_extration_aim = coords_extration_aim[['geometry']]
    
    if coords_extration_aim.shape[0] > 0:
        dist = ckdnearest(coords_distance_raster, coords_extration_aim, "dist_class_"+str(type_num))
        dist_column = dist[["dist_class_"+str(type_num)]]
        coords_distance_raster_result["binary_class_" + str(type_num) + "_" + str(year)] = valueArray / 100
        coords_distance_raster_result["dist_class_" + str(type_num) + "_"] = dist_column
    else:
        print("class_"+str(type_num)+" is not in japan")
    print("dist_class_"+str(type_num))
    type_num += 1

class_0 is not in japan
dist_class_0
class_1 is not in japan
dist_class_1
class_2 is not in japan
dist_class_2
class_3 is not in japan
dist_class_3
class_4 is not in japan
dist_class_4
class_5 is not in japan
dist_class_5
class_6 is not in japan
dist_class_6
class_7 is not in japan
dist_class_7
class_8 is not in japan
dist_class_8
class_9 is not in japan
dist_class_9
class_10 is not in japan
dist_class_10
class_11 is not in japan
dist_class_11
class_12 is not in japan
dist_class_12
class_13 is not in japan
dist_class_13
class_14 is not in japan
dist_class_14
class_15 is not in japan
dist_class_15


Signature: os.mkdir(path, mode=511, *, dir_fd=None)
Docstring:
Create a directory.

If dir_fd is not None, it should be a file descriptor open to a directory,
  and path should be relative; path will then be relative to that directory.
dir_fd may not be implemented on your platform.
  If it is unavailable, using it will raise a NotImplementedError.

The mode argument is ignored on Windows.
Type:      builtin_function_or_method
